checking whether the model loads, have no clue abt dtw and others

In [1]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("kavsss/wav2vec2-tune-timit-asr-for-phoneme")
processor = Wav2Vec2Processor.from_pretrained("kavsss/wav2vec2-tune-timit-asr-for-phoneme")

c:\Users\kavya\anaconda3\envs\accent-capstone\lib\site-packages\transformers\modeling_utils.py:1331: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(re

In [2]:
import torch

In [3]:
from phoneme import extract_needed_data

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
def prepare_dataset(batch):
    #audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(batch["audio_arr"], sampling_rate=16000).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["phoneme"]).input_ids
    return batch

In [5]:
#path = './TIMIT/data/TEST/DR7/FISB0/SA1'
path = './TIMIT/data/TRAIN/DR3/MKXL0/SX195'
audio_arr, line, phoneme = extract_needed_data(path)
batch = {
    "audio_arr": audio_arr,
    "line": line,
    "phoneme": phoneme,
}

In [6]:
batch = prepare_dataset(batch)

In [7]:
batch["input_values"]

array([-0.00615697,  0.00373645, -0.00043596, ...,  0.00068598,
        0.00926793, -0.00015791], dtype=float32)

In [8]:
print(batch["labels"])

[58, 0, 54, 0, 14, 0, 13, 0, 59, 0, 55, 0, 6, 0, 37, 0, 5, 0, 2, 0, 36, 0, 35, 0, 49, 0, 6, 0, 57, 0, 9, 0, 37, 0, 32, 0, 39, 0, 52, 0, 49]


In [9]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cpu").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["pred_phoneme"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

#results = timit["test"].map(map_to_result, remove_columns=timit["test"].column_names)

In [10]:
output = map_to_result(batch)

In [11]:
batch["phoneme"]

'w|uh|dcl|d|y|uw|ax|l|aw|ae|kcl|k|s|ax|v|ay|l|ix|n|tcl|s'

In [12]:
with processor.as_target_processor():
  real = processor(batch["phoneme"]).input_ids

In [13]:
print(real)

[58, 0, 54, 0, 14, 0, 13, 0, 59, 0, 55, 0, 6, 0, 37, 0, 5, 0, 2, 0, 36, 0, 35, 0, 49, 0, 6, 0, 57, 0, 9, 0, 37, 0, 32, 0, 39, 0, 52, 0, 49]


In [14]:
pred_phoneme_style = '|'.join(output["pred_phoneme"].split())

In [16]:
with processor.as_target_processor():
  pred = processor(pred_phoneme_style).input_ids

In [17]:
print(pred)

[58, 0, 54, 0, 14, 0, 13, 0, 59, 0, 55, 0, 6, 0, 37, 0, 5, 0, 2, 0, 36, 0, 35, 0, 49, 0, 6, 0, 57, 0, 9, 0, 37, 0, 32, 0, 39, 0, 52, 0, 49]


In [262]:
#from analysis import compute_alingment, analyze_alingment, calculate_confidence

In [29]:
import numpy as np
from fastdtw import dtw

In [30]:
def compute_alingment(pho1, pho2):
    pho1 = np.array(pho1)
    pho2 = np.array(pho2)
    _, path = dtw(pho1, pho2)#, keep_internals = True)
    return path

def analyze_alingment(path, pho1, pho2):
    matches = []
    mismatches = []
    #for i, j in zip(alingment[1:]):
    #print(list(itertools.chain(alingment[1:])))
    for i, j in path:
        if pho1[i] == pho2[j]:
            matches.append((pho1[i], pho2[j]))
        else:
            mismatches.append((pho1[i], pho2[j]))
    return matches, mismatches

def calculate_confidence(pho1, pho2):
    path = compute_alingment(pho1, pho2)
    matches, mismatches = analyze_alingment(path, pho1, pho2)
    max_distance = max(len(matches), len(mismatches))
    confidence = 1 - (len(mismatches) / max_distance)
    return confidence

In [31]:
path = compute_alingment(np.array(real), np.array(pred))
print(path)

[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (19, 19), (20, 20), (21, 21), (22, 22), (23, 23), (24, 24), (25, 25), (26, 26), (27, 27), (28, 28), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 39), (40, 40)]


In [32]:
match, mismatch = analyze_alingment(path, real, pred)
print(match)

[(58, 58), (0, 0), (54, 54), (0, 0), (14, 14), (0, 0), (13, 13), (0, 0), (59, 59), (0, 0), (55, 55), (0, 0), (6, 6), (0, 0), (37, 37), (0, 0), (5, 5), (0, 0), (2, 2), (0, 0), (36, 36), (0, 0), (35, 35), (0, 0), (49, 49), (0, 0), (6, 6), (0, 0), (57, 57), (0, 0), (9, 9), (0, 0), (37, 37), (0, 0), (32, 32), (0, 0), (39, 39), (0, 0), (52, 52), (0, 0), (49, 49)]


In [33]:
print(mismatch)

[]


In [34]:
confidence = calculate_confidence(real, pred)
print(confidence)

1.0


In [35]:
round(confidence*100, 2)

100.0